# NORMALIZATION

# before Normilizing 
Dynamic Range Compression
Applying dynamic range compression before or after normalization can also help. Dynamic range compression reduces the gap between the loudest and the softest parts of the audio, making it less likely that normalization will introduce noise.

### High Level of Background Noise:
Root Mean Square (RMS) Normalization with with Gain Control  *****

In cases of high background noise, RMS normalization can be effective. This method balances the audio based on the square root of the mean square of the audio signal, taking both signal and noise into account. This type of normalization doesn't overly amplify the noise but ensures that the voice in the recording is sufficiently loud.

### Medium Level of Background Noise:
Peak Normalization with Dynamic Range ***

Dynamic Range Compression
Applying dynamic range compression before or after normalization can also help. Dynamic range compression reduces the gap between the loudest and the softest parts of the audio, making it less likely that normalization will introduce noise.

When you have a moderate amount of background noise, peak normalization could be a good option. It adjusts the gain so that the highest amplitude in the audio reaches a specified level. This ensures that you're not losing any information while preparing the audio for further processing.

### Low Level of Background Noise or Clean Audio:
Z-Score Normalization with Pre-Filtering *

For relatively clean audio, Z-score normalization can be effective. It centers the audio signal around zero and scales it based on its standard deviation. This is particularly useful when the audio is part of a larger dataset with varying quality, as it brings everything to a standard range.

# 1_ check Variation level - later on do it by pieces of audio and First NORM

In [1]:
from scipy.io import wavfile
import numpy as np

def read_and_classify_wav(file_path):
    # ^^^ path ^^^
    sample_rate, data = wavfile.read(file_path)
    
    # Calculate the standard deviation of the audio data
    std_dev = np.std(data)
    
    # Classify the variation based on the standard deviation
    if std_dev < 2000:
        variation = "low_variation"
    elif std_dev < 4000:
        variation = "medium_variation"
    else:
        variation = "high_variation"

    return sample_rate, data, variation

def normalize_audio(data, variation):
    if variation == "low_variation":
        scaling_factor = 0.5
    elif variation == "medium_variation":
        scaling_factor = 0.7
    else: # high_variation
        scaling_factor = 1.0
    
    # Normalize the audio data
    normalized_data = np.int16(data / np.max(np.abs(data)) * 32767 * scaling_factor)
    
    return normalized_data

if __name__ == "__main__":
    file_path = "0_vr_original.wav"    # ^^^ path ^^^
    sample_rate, data, variation = read_and_classify_wav(file_path)
    print(f"Audio variation level: {variation}")

    normalized_data = normalize_audio(data, variation)
    # Save the normalized audio back to a new WAV file
    wavfile.write("1_variation_based_normalized_audio.wav", sample_rate, normalized_data)
    # ^^^ path ^^^

    print("Summary and Conclusions:")
    print("The audio file was successfully read and its amplitude variation level was classified.")
    print(f"The determined level of variation is: {variation}.")
    print("The audio data has been normalized using a scaling factor tailored to this level of variation.")
    print("This approach allows for nuanced normalization, setting the stage for effective further audio processing.")


Audio variation level: high_variation
Summary and Conclusions:
The audio file was successfully read and its amplitude variation level was classified.
The determined level of variation is: high_variation.
The audio data has been normalized using a scaling factor tailored to this level of variation.
This approach allows for nuanced normalization, setting the stage for effective further audio processing.


# 2 _SELECT 2 normalization is dynamic -> try other combinations in the future

In [4]:
import soundfile as sf
import numpy as np
from scipy.signal import butter, lfilter

def rms_normalization(audio_signal, gain=0.9):
    rms_value = np.sqrt(np.mean(np.square(audio_signal)))
    normalized_audio = gain * (audio_signal / rms_value)
    return normalized_audio

def peak_normalization(audio_signal):
    peak_value = np.max(np.abs(audio_signal))
    return audio_signal / peak_value

def z_score_normalization(audio_signal):
    mean = np.mean(audio_signal)
    std = np.std(audio_signal)
    return (audio_signal - mean) / std

def compress_dynamic_range(audio_signal, threshold=0.015, ratio=.002):
    compressed_audio = np.where(np.abs(audio_signal) > threshold,
                                threshold + ((np.abs(audio_signal) - threshold) / ratio),
                                audio_signal)
    return np.sign(audio_signal) * compressed_audio

def butter_lowpass_filter(audio_signal, cutoff_freq, sample_rate):
    b, a = butter(6, cutoff_freq / (0.5 * sample_rate), btype='low')
    return lfilter(b, a, audio_signal)

def main():
    # ^^^ path ^^^
    audio_data, sample_rate = sf.read("1_variation_based_normalized_audio.wav")
    
    print("Choose the type of normalization:")
    print("1: Z-Score Normalization with Pre-Filtering         _for_  *     LOW Level of Background Noise")
    print("2: Peak Normalization with Dynamic Range Compression_for_  ***   MID Level of Background Noise")
    print("3: RMS Normalization with Gain Control              _for_  ***** HIGH Level of Background Noise ")
    

    
    choice = input("Enter the number corresponding to your choice: ")

    if choice == '1':
        filtered_audio = butter_lowpass_filter(audio_data, cutoff_freq=3000, sample_rate=sample_rate)
        normalized_audio = z_score_normalization(filtered_audio)
    elif choice == '2':
        compressed_audio = compress_dynamic_range(audio_data)
        normalized_audio = peak_normalization(compressed_audio)
    elif choice == '3':
        normalized_audio = rms_normalization(audio_data, gain=0.7)
    else:
        print("Invalid choice. Exiting.")
        exit()

    # ^^^ path ^^^
    sf.write("2_dynamic_range_normalized_audio.wav", normalized_audio, sample_rate)
    print("Normalization complete. The file has been saved as '2_dynamic_range_normalized_audio.wav'")

if __name__ == '__main__':
    main()


Choose the type of normalization:
1: Z-Score Normalization with Pre-Filtering         _for_  *     LOW Level of Background Noise
2: Peak Normalization with Dynamic Range Compression_for_  ***   MID Level of Background Noise
3: RMS Normalization with Gain Control              _for_  ***** HIGH Level of Background Noise 
Enter the number corresponding to your choice: 2
Normalization complete. The file has been saved as '2_dynamic_range_normalized_audio.wav'
